# ONNX to TFLite Conversion using onnx2tf

This notebook demonstrates converting an ONNX model to TFLite format using the modern `onnx2tf` tool.

`onnx2tf` is specifically designed for ONNX → TFLite conversion and works with current TensorFlow versions.

## 1. Setup and Import Libraries

In [ ]:
import os
import tensorflow as tf
import numpy as np
import sys 

print(f"TensorFlow version: {tf.__version__}")
print(f"Python version: {sys.version}")

TensorFlow version: 2.20.0
Python version: 3.10.19 (main, Oct 14 2025, 21:43:20) [Clang 20.1.4 ]


## 2. Set Model Path

In [2]:
# Set path to your ONNX model
onnx_path = 'segment_model.onnx'

# Check that the file exists
assert os.path.exists(onnx_path), f"ONNX model not found at {onnx_path}"
print(f"✓ Found ONNX model at: {onnx_path}")

✓ Found ONNX model at: segment_model.onnx


## 3. Convert ONNX to TFLite

The `onnx2tf` tool performs the conversion in a single step, going directly from ONNX to TFLite without intermediate formats.

### Conversion Options:
- `-i`: Input ONNX model path
- `-o`: Output directory for converted model
- `-osd`: Output SavedModel directory
- `-coion`: Copy ONNX input order to output (maintains input shape compatibility)

In [3]:
# Run onnx2tf conversion
!onnx2tf -i segment_model.onnx -o saved_model -osd

Traceback (most recent call last):
  File "/Users/niner9/Documents/GitHub/URD-2_BlackPigProject/.venv/bin/onnx2tf", line 3, in <module>
    from onnx2tf import main
  File "/Users/niner9/Documents/GitHub/URD-2_BlackPigProject/.venv/lib/python3.10/site-packages/onnx2tf/__init__.py", line 1, in <module>
    from onnx2tf.onnx2tf import convert, main
  File "/Users/niner9/Documents/GitHub/URD-2_BlackPigProject/.venv/lib/python3.10/site-packages/onnx2tf/onnx2tf.py", line 29, in <module>
    import tf_keras
ModuleNotFoundError: No module named 'tf_keras'


## 4. Verify Output Files

The conversion should produce:
- `saved_model/` directory with SavedModel format
- `saved_model/model_float32.tflite` - the TFLite model

In [ ]:
# Check output files
output_dir = 'saved_model'
tflite_path = os.path.join(output_dir, 'model_float32.tflite')

if os.path.exists(tflite_path):
    file_size = os.path.getsize(tflite_path) / (1024 * 1024)  # Convert to MB
    print(f"✓ TFLite model created successfully!")
    print(f"  Path: {tflite_path}")
    print(f"  Size: {file_size:.2f} MB")
else:
    print(f"✗ TFLite model not found at {tflite_path}")

# List all files in output directory
print("\nOutput directory contents:")
for root, dirs, files in os.walk(output_dir):
    level = root.replace(output_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

## 5. Inspect TFLite Model Details

In [ ]:
# Load the TFLite model and inspect its details
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("=== Input Details ===")
for i, detail in enumerate(input_details):
    print(f"\nInput {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Data type: {detail['dtype']}")

print("\n=== Output Details ===")
for i, detail in enumerate(output_details):
    print(f"\nOutput {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Data type: {detail['dtype']}")

## 6. Test Inference (Optional)

Run a simple test inference to verify the model works.

In [ ]:
# Create dummy input data based on the model's expected input shape
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

print(f"Creating test input with shape: {input_shape}, dtype: {input_dtype}")

# Generate random test data
test_input = np.random.randn(*input_shape).astype(input_dtype)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], test_input)

# Run inference
interpreter.invoke()

# Get output
output_data = interpreter.get_tensor(output_details[0]['index'])

print(f"\n✓ Inference successful!")
print(f"  Output shape: {output_data.shape}")
print(f"  Output dtype: {output_data.dtype}")
print(f"  Output range: [{output_data.min():.4f}, {output_data.max():.4f}]")

## 7. Copy TFLite Model to Root Directory (Optional)

In [ ]:
import shutil

# Copy the TFLite model to a more accessible location
output_tflite_path = 'segment_model.tflite'
shutil.copy(tflite_path, output_tflite_path)

print(f"✓ TFLite model copied to: {output_tflite_path}")

## Summary

This notebook successfully converted an ONNX model to TFLite format using `onnx2tf`.

The resulting TFLite model can be deployed on:
- Android devices
- iOS devices
- Embedded systems
- Edge devices

### Next Steps:
1. Optimize the model further (quantization, pruning)
2. Integrate into your mobile application
3. Benchmark performance on target device